# Notebook de conception de Notebook

Ce Notebook .Net interactive a pour objectif de permettre la création assistée d'autres notebooks .Net interactive en confiant le soin à ChatGPT d'analyser et de proposer des modifications d'une version courante, et en prenant en charge la mise à jour et l'exécution des mises à jour en function calling Open AI grâce à l'API .Net interactive. 


### 1. Initialisation

On installe des packages pour la manipulation de notebook et pour l'orchestration de LLMs.

In [1]:
// #r "nuget: Microsoft.DotNet.Interactive, *-*"
#r "nuget: Microsoft.DotNet.Interactive.CSharp, *-*"
#r "nuget: Microsoft.DotNet.Interactive.Documents, *-*"

#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview"

Installed Packages Microsoft.DotNet.Interactive.CSharp, 1.0.0-beta.24229.4 Microsoft.DotNet.Interactive.Documents, 1.0.0-beta.24229.4 Microsoft.Extensions.Logging, 8.0.0 Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview

- **Imports des espaces de noms**

On prend soin de distinguer le kernel d'exécution de notebook .Net interactive, et le kernel de semantic-kernel.

In [2]:
  using Microsoft.DotNet.Interactive;
  using Microsoft.SemanticKernel;
  using Microsoft.SemanticKernel.Planning;
  using Microsoft.SemanticKernel.Connectors.OpenAI;
  
  using System;
  using System.IO;
  using System.Threading.Tasks;

  using SKernel = Microsoft.SemanticKernel.Kernel;
  using IKernel = Microsoft.DotNet.Interactive.Kernel;

- **Configurez l'authentification des services semantic-kernel**

Créer au besoin le fichier config/settings.json pour la config semantic-kernel

In [3]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

- **Création d'un logger dédié**

On crée un logger qui s'affichera en cellule de sortie

In [4]:
using Microsoft.Extensions.Logging;
using System;

public class DisplayLogger : ILogger, ILoggerFactory
{
    private readonly string _categoryName;
    private readonly LogLevel _logLevel;

    public DisplayLogger(string categoryName, LogLevel logLevel)
    {
        _categoryName = categoryName;
        _logLevel = logLevel;
    }

    public IDisposable BeginScope<TState>(TState state) => this;

    public bool IsEnabled(LogLevel logLevel) => logLevel >= _logLevel;

    public void Log<TState>(LogLevel logLevel, EventId eventId, TState state, Exception exception, Func<TState, Exception, string> formatter)
    {
        if (!IsEnabled(logLevel))
        {
            return;
        }

        var logEntry = $"[{logLevel}] {_categoryName} - {formatter(state, exception)}";
        if (exception != null)
        {
            logEntry += Environment.NewLine + exception;
        }

        display(logEntry);
    }

        /// <inheritdoc/>
        public void Dispose()
        {
        // This class is marked as disposable to support the BeginScope method.
        // However, there is no need to dispose anything.
        }

    public ILogger CreateLogger(string categoryName) => this;

        public void AddProvider(ILoggerProvider provider) => throw new NotSupportedException();

}

public class DisplayLoggerProvider : ILoggerProvider
{
    private readonly LogLevel _logLevel;

    public DisplayLoggerProvider(LogLevel logLevel)
    {
        _logLevel = logLevel;
    }

    public ILogger CreateLogger(string categoryName)
    {
        return new DisplayLogger(categoryName, _logLevel);
    }

    public void Dispose() { }
}


- **Initialisation semantic-kernel:**

On utilise les informations de configuration de semantic-kernel pour créer un kernel. On le dotera par la suite de plugins pour l'utiliser dans un planner.

In [5]:
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;
using System;

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// // Configure custom logger
// var loggerFactory = LoggerFactory.Create(builder =>
// {
//     builder.AddProvider(new DisplayLoggerProvider(LogLevel.Information));
// });

var builder = SKernel.CreateBuilder();

builder.Services.AddLogging(loggingBuilder =>
{
    loggingBuilder.AddProvider(new DisplayLoggerProvider(LogLevel.Information));
});

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

// builder.WithLoggerFactory(loggerFactory);

var semanticKernel = builder.Build();

display("Kernel and Semantic Kernel initialized.");


Kernel and Semantic Kernel initialized.

### 2. Mode de Fourniture des Informations

On permet à l'utilisateur de saisir les informations décrivant la tâche à accomplir dans le notebook de travail de plusieurs façons différentes.

In [6]:
public enum InformationMode
{
    Variable,
    Prompt,
    File
}

var mode = InformationMode.Variable; // Changez cette valeur pour tester les différents modes

#### Reccueil des informations

Selon le mode de fourniture des informations choisi, on récupère la tâche à accomplir dans le notebook de travail.

In [7]:
var infoCollectionDisplay = display("Collecte d'informations en cours...");

string taskDescription = "Créer un notebook permettant de requêter DBPedia en utilisant RDF.Net";

if (mode == InformationMode.Variable)
{
    display("Utilisation de la variable pour la description de la tâche.");
}
else if (mode == InformationMode.Prompt)
{
    var questions = new[]
    {
        "Bonjour! Veuillez fournir une brève description de la tâche à accomplir.",
        "Quels sont les principaux objectifs de cette tâche?",
        "Y a-t-il des contraintes ou des conditions spécifiques à prendre en compte?",
        "Des informations supplémentaires que vous souhaitez ajouter?"
    };

    taskDescription = string.Empty;
    foreach (var question in questions)
    {
        var response = await IKernel.GetInputAsync(question);
        taskDescription += $"{question}\\n{response}\\n\\n";
    }
}


display("Informations recueillies :\\n" + taskDescription);

Collecte d'informations en cours...

Utilisation de la variable pour la description de la tâche.

Informations recueillies :\nCréer un notebook permettant de requêter DBPedia en utilisant RDF.Net

### 3. Personnalisation du Notebook de Travail

On charge un notebook template contenant des parties de Markdown et de code à compléter, et on injecte la tâche dans la partie descriptive en entête du notebook .Net interactive.

In [8]:


var notebookTemplatePath = "./Workbook-Template.ipynb";
var notebookPath = @$"./Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}.ipynb";
var notebookOutputPath = @$"./Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}-output.ipynb";

string notebookContent = File.ReadAllText(notebookTemplatePath);

display(@"Personnalisation du notebook {notebookPath} en cours...");

notebookContent = notebookContent.Replace("{{TASK_DESCRIPTION}}", taskDescription);

File.WriteAllText(notebookPath, notebookContent);
display($"Notebook personnalisé prêt à l'exécution");

Personnalisation du notebook {notebookPath} en cours...

Notebook personnalisé prêt à l'exécution

### 4. Boucle Récurrente pour l'Exécution du Notebook

- **Classe en charge de charger, d'exécuter et renvoyer un le résultat d'un notebook**

In [9]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Documents;
using DKernelInfo = Microsoft.DotNet.Interactive.Documents.KernelInfo;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using System.Reactive.Linq;
using System.Text;
using System.Threading;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

public class NotebookExecutor
{
    public static string PlainTextValue(DisplayEvent @event)
    {
        return @event.FormattedValues.FirstOrDefault()?.Value ?? string.Empty;
    }

    private readonly CompositeKernel _kernel;

    public NotebookExecutor(CompositeKernel kernel)
    {
        _kernel = kernel;
    }

    public async Task<InteractiveDocument> RunNotebookAsync(
        InteractiveDocument notebook,
        IDictionary<string, string>? parameters = null,
        CancellationToken cancellationToken = default)
    {
        var notebookExecutionDisplay = display("Exécution du notebook en cours...");
        notebookExecutionDisplay.Update("Début de l'exécution du notebook...");

        var resultDocument = new InteractiveDocument();

        if (parameters is not null)
        {
            parameters = new Dictionary<string, string>(parameters, StringComparer.InvariantCultureIgnoreCase);
        }

        var kernelInfoCollection = CreateKernelInfos(_kernel);
        var lookup = kernelInfoCollection.ToDictionary(k => k.Name, StringComparer.OrdinalIgnoreCase);

        foreach (var element in notebook.Elements)
        {
            if (lookup.TryGetValue(element.KernelName!, out var kernelInfo) &&
                StringComparer.OrdinalIgnoreCase.Equals(kernelInfo.LanguageName, "markdown"))
            {
                var formattedValue = new FormattedValue("text/markdown", element.Contents);
                var displayValue = new DisplayValue(formattedValue);
                display($"Affichage du markdown: {element.Contents}");
                await _kernel.SendAsync(displayValue);
            }
            else
            {
                var submitCode = new SubmitCode(element.Contents, element.KernelName);
                display($"Envoi du code au kernel {element.KernelName}: {element.Contents}");
                await _kernel.SendAsync(submitCode);
            }
        }

        var defaultKernelName = _kernel.DefaultKernelName;
        var defaultKernel = _kernel.ChildKernels.SingleOrDefault(k => k.Name == defaultKernelName);
        var languageName = defaultKernel?.KernelInfo.LanguageName ?? notebook.GetDefaultKernelName() ?? "C#";

        resultDocument.Metadata["kernelspec"] = new Dictionary<string, object>
        {
            { "name", defaultKernel?.Name ?? "csharp" },
            { "language", languageName }
        };

        notebookExecutionDisplay.Update("Exécution du notebook terminée.");

        return resultDocument;
    }

    private KernelInfoCollection CreateKernelInfos(CompositeKernel kernel)
    {
        KernelInfoCollection kernelInfos = new();

        foreach (var childKernel in kernel.ChildKernels)
        {
            kernelInfos.Add(new DKernelInfo(childKernel.Name, languageName: childKernel.KernelInfo.LanguageName, aliases: childKernel.KernelInfo.Aliases));
        }

        if (!kernelInfos.Contains("markdown"))
        {
            kernelInfos = kernelInfos.Clone();
            kernelInfos.Add(new DKernelInfo("markdown", languageName: "Markdown"));
        }

        return kernelInfos;
    }
}


- **Classe du plugin semantic-kernel pour function calling**

Ce plugin permet à chat GPT de mettre à jour un notebook, et lui renvoie la nouvelle version rééxécutée.

In [10]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
using Microsoft.DotNet.Interactive.Documents;
using Microsoft.DotNet.Interactive.Documents.Jupyter;
using System.Threading.Tasks;
using System.IO;
using System.ComponentModel;
using Microsoft.DotNet.Interactive.CSharp;
using System.Reflection;

public class WorkbookInteraction
{

    private readonly string _notebookPath ;
    private readonly NotebookExecutor _executor;
    private object _workbookDisplay;
    private MethodInfo _updateDisplayMethod = typeof(DisplayedValue).GetMethod("Update");

     private readonly ILogger _logger;

    public WorkbookInteraction(string notebookPath, ILogger logger)
    {
        _notebookPath = notebookPath;
        var compositeKernel = new CompositeKernel
        {
            new CSharpKernel()
        };
        _executor = new NotebookExecutor(compositeKernel);
        _logger = logger;
    }

    private void DisplayWorkbook(string displayContent)
    {
        if (_workbookDisplay is null)
        {
            _workbookDisplay = display(displayContent);
        }

        _updateDisplayMethod.Invoke(_workbookDisplay, new object[] { displayContent });
    }

    [KernelFunction]
    [Description("Runs an updated version of the workbook and returns the notebook with output cells")]
    public async Task<string> UpdateWorkbook(
        [Description("the new version of the workbook in ipynb json format")] string updatedWorkbook)
    {
        var updateDisplay = display("Appel en function calling à UpdateWorkbook...");
        DisplayWorkbook(updatedWorkbook);
        File.WriteAllText(_notebookPath, updatedWorkbook);
        _logger.LogInformation("UpdateWorkbook called with updated workbook content.");

        var notebook = await InteractiveDocument.LoadAsync(new FileInfo(_notebookPath));
        var resultDocument = await _executor.RunNotebookAsync(notebook);
        var outputJson = resultDocument.ToJupyterJson();
        // File.WriteAllText(notebookOutputPath, outputJson );
        updateDisplay.Update("Appel à UpdateWorkbook terminé, renvoi du workbook rééxécuté...");
        DisplayWorkbook(outputJson);
        return outputJson;
        
    }

    // [KernelFunction]
    // [Description("Returns the final answer if last version of the workbook allowed to conclude the task.")]
    // public string SendFinalAnswer([Description("The final answer to the task at hand")] string answer)
    // {
    //     var resultDisplay = display("Final Answer: " + answer);
    //     return answer;
    // }
}


- **Classe en charge de l'exécution du planner:**

In [11]:
#pragma warning disable SKEXP0060

public class NotebookUpdater
{

    private readonly FunctionCallingStepwisePlanner _planner;
    private readonly SKernel _semanticKernel;
    private readonly string _notebookPath;

    private readonly ILogger _logger;

    public NotebookUpdater(SKernel semanticKernel, string notebookPath, ILogger logger)
    {
        _semanticKernel = semanticKernel;
        _notebookPath = notebookPath;
        var options = new FunctionCallingStepwisePlannerOptions
        {
            MaxIterations = 5,
            ExecutionSettings = new OpenAIPromptExecutionSettings { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions }
        };
        _planner = new FunctionCallingStepwisePlanner(options);
        _logger = logger;
        var workbookInteraction = new WorkbookInteraction(notebookPath, _logger);

        
        _semanticKernel.ImportPluginFromObject(workbookInteraction);
    }

    public async Task<string> UpdateNotebook()
    {
        display("Lecture du contenu du notebook...");
        var notebookJson = File.ReadAllText(_notebookPath);

        var updateDisplay = display("Appel de ChatGPT avec le workbook initialisé...");
        
        var plannerPrompt = $"Analyse le notebook suivant qui contient la description de son objectif, utilise le function calling avec la méthode UpdateWorkbook pour éditer et réexécuter le notebook jusqu'à ce qu'il donne satisfaction et renvoie la réponse finale.\\n\\n{notebookJson}";
        display("Envoi du prompt au planner...");
        
        var result = await _planner.ExecuteAsync(_semanticKernel, plannerPrompt);
        
        updateDisplay.Update("Notebook mis à jour avec succès.");

        return result.FinalAnswer;
        
    }
}


### 5. Exécution et Mise à Jour Itérative

In [12]:
var logger = new DisplayLogger("NotebookUpdater", LogLevel.Information);

display("Création de l'instance NotebookUpdater...");
var updater = new NotebookUpdater(semanticKernel, notebookPath, logger);

display("Appel à UpdateNotebook...");
var response = await updater.UpdateNotebook();

display($"Résultat de l'exécution du notebook :\n{response}");


Création de l'instance NotebookUpdater...

Appel à UpdateNotebook...

Lecture du contenu du notebook...

Notebook mis à jour avec succès.

Envoi du prompt au planner...

...

[Information] UpdateWorkbook - Function completed. Duration: 0.3399289s

[Information] Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIChatCompletionService - Prompt tokens: 3315. Completion tokens: 59. Total tokens: 3374.

[Information] Microsoft.SemanticKernel.Planning.FunctionCallingStepwisePlanner - Plan executed successfully.

[Information] Microsoft.SemanticKernel.Planning.FunctionCallingStepwisePlanner - Plan execution duration: 34.9479294s.

Résultat de l'exécution du notebook :
Le notebook a été mis à jour mais aucune cellule n'a été retournée avec des résultats. Veuillez vérifier la version mise à jour du notebook pour vous assurer que toutes les étapes ont été correctement ajoutées.

### Conclusion